In [1]:
%load_ext autoreload
%autoreload 2

from boolrank import DualSiglip2Model

# model = DualSiglip2Model('BAAI/llm-embedder')
model = DualSiglip2Model('BAAI/bge-small-en-v1.5')
# model = DualSiglip2Model('dmis-lab/biobert-v1.1')
_ = model.eval()

In [2]:
import pandas as pd
from collections import defaultdict
import torch

group_keys: list[str] = ["id", "model"]
main_name = group_keys[-1]
res = defaultdict(lambda: defaultdict(list))
df = pd.read_json("data/combined_outputs.jsonl", lines=True)
byid = df.sort_values("f3").groupby(group_keys)
prompt_data = list(map(lambda tpl: tpl[1], byid))

group = prompt_data[0]
print(group.f3)
logits = model(list(group["generated_query"].values), group["topic"].iloc[0])["logits"]
tensor = logits.squeeze().cpu()
print(tensor)
torch.argsort(tensor)

1266    0.0000
1380    0.0277
1283    0.0308
1179    0.0314
1219    0.0435
1322    0.0741
1139    0.0856
Name: f3, dtype: float64
tensor([0.7545, 0.7496, 0.7905, 0.7594, 0.7945, 0.7015, 0.7695],
       grad_fn=<ToCopyBackward0>)


tensor([5, 1, 0, 3, 6, 2, 4])

In [3]:
from evaluation import evaluate_on_generated

evaluate_on_generated(model)

,model,spearman,pearson,f3_variance
0,HuggingfaceH4,-0.1954,-0.2592,0.0064
1,gpt-3.5-turbo-0125,-0.2898,-0.3060,0.0055
2,gpt-3.5-turbo-1106,-0.1841,-0.2397,0.0049
3,gpt-4-1106-preview,-0.5975,-0.5472,0.0090
4,gpt-4o-mini,-0.4350,-0.4537,0.0065
5,meta-llama,-0.2317,-0.1849,0.0085
6,mistralai,-0.2555,-0.2449,0.0076
7,o1-2024-12-17,0.0692,0.1410,0.0048
8,open-mistral-7b,-0.2850,-0.3664,0.0071
9,open-mixtral-8x7b,-0.1733,-0.2043,0.0095


In [4]:
evaluate_on_generated(model, ["prompt_id", "id"])

c:\Users\Simon\Documents\Ordner\Uni\Master\25-Sommer\Research Project\project-doehl-boolean-clip\evaluation.py:204: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  avg_row[main_name] = "Average"


,id,spearman,pearson,f3_variance
0,1,-0.2592,-0.1923,0.0005
1,2,-0.0918,-0.0685,0.0003
2,3,0.0364,0.1085,0.0187
3,4,-0.1204,-0.1783,0.0017
4,6,-0.5429,-0.5582,0.0106
5,7,0.1364,0.3050,0.0017
6,8,0.1639,0.1398,0.0005
7,10,0.2864,0.3269,0.0137
8,11,0.0663,0.1177,0.0106
9,12,0.1440,0.1257,0.0131


In [5]:
model.load(r"models\clip\bge-small-en-v1.5\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
# model.load(r"models\clip\biobert-v1.1\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-14110\model.safetensors")
evaluate_on_generated(model)

,model,spearman,pearson,f3_variance
0,HuggingfaceH4,0.0566,0.0422,0.0064
1,gpt-3.5-turbo-0125,-0.0750,-0.0062,0.0055
2,gpt-3.5-turbo-1106,0.0188,0.0130,0.0049
3,gpt-4-1106-preview,0.0956,0.0352,0.0090
4,gpt-4o-mini,-0.0564,-0.0541,0.0065
5,meta-llama,-0.0855,-0.1225,0.0085
6,mistralai,0.0606,0.1338,0.0076
7,o1-2024-12-17,-0.0462,0.0022,0.0048
8,open-mistral-7b,-0.0407,0.0092,0.0071
9,open-mixtral-8x7b,-0.0145,-0.0913,0.0095


In [6]:
evaluate_on_generated(model, ["prompt_id", "id"])

c:\Users\Simon\Documents\Ordner\Uni\Master\25-Sommer\Research Project\project-doehl-boolean-clip\evaluation.py:204: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  avg_row[main_name] = "Average"


,id,spearman,pearson,f3_variance
0,1,-0.1850,0.1005,0.0005
1,2,-0.2000,-0.3001,0.0003
2,3,0.0153,-0.0537,0.0187
3,4,-0.1592,-0.2724,0.0017
4,6,0.4000,0.4482,0.0106
5,7,0.0990,0.2526,0.0017
6,8,-0.4204,-0.2520,0.0005
7,10,-0.0276,0.0434,0.0137
8,11,0.0323,0.0732,0.0106
9,12,0.1857,0.2393,0.0131


In [7]:
model.load(r"models\clip\bge-small-en-v1.5\b16_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
evaluate_on_generated(model)

,model,spearman,pearson,f3_variance
0,HuggingfaceH4,-0.1414,-0.1765,0.0064
1,gpt-3.5-turbo-0125,-0.2327,-0.2601,0.0055
2,gpt-3.5-turbo-1106,-0.1586,-0.1655,0.0049
3,gpt-4-1106-preview,-0.4176,-0.4181,0.0090
4,gpt-4o-mini,-0.3727,-0.4172,0.0065
5,meta-llama,-0.1234,-0.1882,0.0085
6,mistralai,0.0441,0.0251,0.0076
7,o1-2024-12-17,0.0718,0.1081,0.0048
8,open-mistral-7b,-0.1810,-0.2977,0.0071
9,open-mixtral-8x7b,-0.1084,-0.1031,0.0095


In [8]:
evaluate_on_generated(model, ["prompt_id", "id"])

c:\Users\Simon\Documents\Ordner\Uni\Master\25-Sommer\Research Project\project-doehl-boolean-clip\evaluation.py:204: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  avg_row[main_name] = "Average"


,id,spearman,pearson,f3_variance
0,1,-0.0830,0.0357,0.0005
1,2,-0.1551,-0.2248,0.0003
2,3,0.2612,0.3274,0.0187
3,4,-0.0990,-0.2189,0.0017
4,6,-0.1122,-0.2347,0.0106
5,7,0.3170,0.3819,0.0017
6,8,-0.0861,0.0214,0.0005
7,10,0.3180,0.3284,0.0137
8,11,0.2398,0.2585,0.0106
9,12,0.1167,0.0745,0.0131
